<a href="https://githubtocolab.com/gee-community/geemap/blob/master/docs/notebooks/118_download_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

**Download images directly to a local computer**

Uncomment the following line to install [geemap](https://geemap.org) if needed.

In [ ]:
# !pip install -U geemap

In [1]:
import os
import ee
import geemap
ee.Authenticate()
ee.Initialize(project='ee-eslamelnahas-jupyter')

## Download image by tile

In [3]:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)


dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2020-01-01', '2020-12-30')
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
    .map(mask_s2_clouds)
    .mean()
)
image = dataset.select(["B4", "B3", "B2"])
visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

m = geemap.Map()
m.add_layer(image, visualization, 'RGB')
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [4]:
# out_dir = os.path.expanduser("~/Downloads")
# m.user_roi.getInfo()

In [5]:
region = ee.Geometry.BBox(32.639008, 30.929949, 33.567352, 31.285043)
m.centerObject(region)

In [6]:
# features = geemap.fishnet(region, rows=2, cols=2,delta=1.0)
# m.addLayer(features, {}, "Grids")

In [16]:
features = geemap.fishnet(region,h_interval=0.2, v_interval=0.2,delta=1.0)
m.addLayer(features, {}, "Grids")

In [18]:
# image = dataset.mean()

In [20]:
geemap.download_ee_image_tiles(
   image , features, out_dir="tiles/tile_1", prefix="Sentinel_", crs="EPSG:4326", scale=10
)

Sentinel_01.tif: |                                                    | 0.00/59.6M (raw) [  0.0%] in 00:00 (et…

There is no STAC entry for: None


Sentinel_02.tif: |                                                    | 0.00/59.5M (raw) [  0.0%] in 00:00 (et…

Sentinel_03.tif: |                                                    | 0.00/59.5M (raw) [  0.0%] in 00:00 (et…

Sentinel_04.tif: |                                                    | 0.00/59.6M (raw) [  0.0%] in 00:00 (et…

...

Sentinel_07.tif: |                                                    | 0.00/59.5M (raw) [  0.0%] in 00:00 (et…

Sentinel_08.tif: |                                                    | 0.00/59.5M (raw) [  0.0%] in 00:00 (et…

Sentinel_09.tif: |                                                    | 0.00/59.5M (raw) [  0.0%] in 00:00 (et…

Sentinel_10.tif: |                                                    | 0.00/59.5M (raw) [  0.0%] in 00:00 (et…

Downloaded 10 tiles in 825.4777252674103 seconds.
